# 大模型学习day3
今天开始换个方式学习，使用cursor帮我生成一个项目，边教我大模型应用中技术比如Langchain，边完善这个工程项目，我使用cursor直接生成了python web的框架，fastAPI+postgres数据库，页面和后端尽可能使用cursor生成，这些对我没有太大的意义。除此之外还生成了CI/CD和dockerfile，这部分可以学习一下。

## 一、RAG最小Demo（Langchain + 本地知识库 + 大模型）
核心流程：
- 用户输入问题
- 检索知识库中相关文档片段
- 将检索结果与用户问题拼接，作为Prompt输入大模型
- 大模型生成最终答案

下面实现RAG Demo代码（可直接在Python脚本中运行），先安装langchain, sentence-transformers, milvus, openai等依赖。

In [ ]:
#  查看当前解释器目录 排查jupyter环境问题 在此之前先创建虚拟环境
import sys
print(sys.executable)

### 1.先安装依赖

In [ ]:


! pip install langchain sentence-transformers pymilvus openai langchain_openai
! pip install -U langchain-community
! pip install dashscope
! pip install faiss-cpu



In [ ]:
# 单独执行下确认依赖已经下载
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Milvus
from langchain.llms import OpenAI  # 你可以换成ChatGLM、Qwen等
from langchain.chains import RetrievalQA

### 2.测试下是否能连接到大模型

In [ ]:
# 确认能加载大模型 这是openai的sdk
import os
from openai import OpenAI


client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    #打印环境变量
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

completion = client.chat.completions.create(
    # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    model="qwen-plus",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "你是谁？"},
    ],
    # Qwen3模型通过enable_thinking参数控制思考过程（开源版默认True，商业版默认False）
    # 使用Qwen3开源版模型时，若未启用流式输出，请将下行取消注释，否则会报错
    # extra_body={"enable_thinking": False},
)
print(completion.model_dump_json())

In [ ]:
# 使用langchain加载大模型
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
import os

api_key=os.getenv("DASHSCOPE_API_KEY")
base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"

llm = ChatOpenAI(
    model="qwen-plus",
    temperature=0,
    api_key=api_key, 
    base_url=base_url)

prompt = PromptTemplate.from_template("你是一个起名字大师，请模仿实例起三个{country}名字，比如男孩名字经常叫做{boy_name}，女孩名字经常叫做{girl_name}")
message = prompt.format(country="中国特色的", boy_name="高义", girl_name="白洁")

print(message)
response = llm.invoke(message)
print(response.content)

### 3.RAG的最小demo

In [ ]:
from langchain.embeddings import DashScopeEmbeddings
from langchain.vectorstores import FAISS  # 推荐本地开发用FAISS
from langchain.llms import OpenAI  # 你可以换成Qwen的Langchain适配器
from langchain.chains import RetrievalQA
from pydantic import SecretStr


# 1. 设置阿里百炼 API Key
import os
key = os.getenv("DASHSCOPE_API_KEY")
# print(os.getenv("DASHSCOPE_API_KEY"))
if key is not None:
    os.environ["DASHSCOPE_API_KEY"] = key
else:
    raise ValueError("环境变量 DASHSCOPE_API_KEY 未设置！")

# 2. 加载阿里百炼 Embedding
embeddings = DashScopeEmbeddings(model="text-embedding-v4")  # 或 text-embedding-v4，具体看你key支持的版本

# 3. 构建本地FAISS向量库（假设你已有文档数据）
from langchain.schema import Document
docs = ["Langchain是一个用于构建RAG应用的框架。", "RAG结合了检索和生成模型。"]
documents = [Document(page_content=doc) for doc in docs]
vector_db = FAISS.from_documents(documents, embeddings)

# 4. 加载大模型（如用Qwen/通义千问，可用Langchain适配器，暂用OpenAI做演示）
llm = ChatOpenAI(
    model="qwen-plus",
    temperature=0,
    api_key=SecretStr(key), 
    base_url=base_url)

# 5. 构建RAG链
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_db.as_retriever(),
    return_source_documents=True
)

# 6. 用户提问
question = "Langchain的RAG原理是什么？"
result = qa({"query": question})

print("答案：", result["result"])
print("参考片段：", [doc.page_content for doc in result["source_documents"]])